In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect('../all_variants.sqlite')
conn.executescript('''\
PRAGMA cache_size=-4192000;
PRAGMA temp_store=MEMORY;
PRAGMA journal_mode=OFF;
''')

## Filter only the samples shared by GDC and MC3

In [3]:
c = conn.execute('''SELECT DISTINCT tumor_sample_barcode FROM mc3_selected''')
mc3_samples = [t[0] for t in c.fetchall()]
c = conn.execute('''SELECT DISTINCT tumor_sample_barcode FROM gdc_grouped_callers''')
gdc_samples = [t[0] for t in c.fetchall()]
print(f'MC3: {len(mc3_samples)}, GDC: {len(gdc_samples)} samples')

gdc_only_samples = set(gdc_samples) - set(mc3_samples)

MC3: 1763, GDC: 1821 samples


In [4]:
_gdc_samples_sql_literal = ','.join(f"'{sample}'" for sample in gdc_only_samples)
c = conn.execute(f'''\
CREATE VIEW IF NOT EXISTS gdc_grp_shared_samples AS
SELECT * FROM gdc_grouped_callers
WHERE tumor_sample_barcode NOT IN ({_gdc_samples_sql_literal})
''')

In [5]:
df_variant_class = pd.merge(
    pd.read_sql('''\
SELECT variant_classification, count(*) AS 'mc3_count'
FROM mc3_selected
GROUP BY variant_classification
ORDER BY mc3_count DESC
''', conn),
    pd.read_sql('''\
SELECT variant_classification, count(*) AS 'gdc_count'
FROM gdc_grp_shared_samples
GROUP BY variant_classification
ORDER BY gdc_count DESC
''', conn),
    how='outer',
    on='variant_classification'
)

df_variant_class = df_variant_class.fillna(0).assign(
    mc3_count = lambda df: df.mc3_count.astype(int),
    gdc_count = lambda df: df.gdc_count.astype(int)
)

In [6]:
df_variant_class

,variant_classification,mc3_count,gdc_count
0,Missense_Mutation,232272,242905
1,Silent,90308,89564
2,3'UTR,31469,40250
3,Frame_Shift_Del,24255,23886
4,Nonsense_Mutation,17785,19958
5,Intron,14202,24839
6,5'UTR,10828,11915
7,Frame_Shift_Ins,6354,10887
8,Splice_Site,6303,5979
9,RNA,5230,8219


In [7]:
df_variant_class.sum()

variant_classification    Missense_MutationSilent3'UTRFrame_Shift_DelNon...
mc3_count                                                            445856
gdc_count                                                            495363
dtype: object

In [8]:
df_per_tx_sample_variant_type = pd.read_sql('''\
WITH gdc_by_tx AS (
    SELECT hugo_symbol, transcript_id, variant_classification, tumor_sample_barcode, count(*) AS 'gdc_count'
    FROM gdc_grp_shared_samples
    GROUP BY hugo_symbol, transcript_id, variant_classification, tumor_sample_barcode
), mc3_by_tx AS (
    SELECT hugo_symbol, transcript_id, variant_classification, tumor_sample_barcode, count(*) AS 'mc3_count'
    FROM mc3_selected
    GROUP BY hugo_symbol, transcript_id, variant_classification, tumor_sample_barcode
)
SELECT hugo_symbol, transcript_id, variant_classification, tumor_sample_barcode, gdc_count, mc3_count FROM gdc_by_tx
LEFT JOIN mc3_by_tx 
USING (hugo_symbol, transcript_id, variant_classification, tumor_sample_barcode)
UNION ALL
SELECT hugo_symbol, transcript_id, variant_classification, tumor_sample_barcode, gdc_count, mc3_count FROM mc3_by_tx
LEFT JOIN gdc_by_tx
USING (hugo_symbol, transcript_id, variant_classification, tumor_sample_barcode)
WHERE gdc_by_tx.gdc_count IS NULL
''', conn)

In [9]:
df_per_tx_sample_variant_type = (
    df_per_tx_sample_variant_type
    .fillna(0)
    .assign(
        gdc_count=lambda df: df.gdc_count.astype(int),
        mc3_count=lambda df: df.mc3_count.astype(int))
    .assign(
        diff=lambda df: df.gdc_count-df.mc3_count
    )
    .sort_values(['diff'], ascending=False)
)

In [10]:
df_per_tx_sample_variant_type.head()

,hugo_symbol,transcript_id,variant_classification,tumor_sample_barcode,gdc_count,mc3_count,diff
420752,TTN,ENST00000591111,Missense_Mutation,TCGA-CA-6717-01A-11D-1835-10,29,0,29
420635,TTN,ENST00000591111,Missense_Mutation,TCGA-AA-A010-01A-01D-A17O-10,29,0,29
420634,TTN,ENST00000591111,Missense_Mutation,TCGA-AA-A00N-01A-02D-A17O-10,28,0,28
420665,TTN,ENST00000591111,Missense_Mutation,TCGA-AN-A046-01A-21W-A050-09,28,0,28
420702,TTN,ENST00000591111,Missense_Mutation,TCGA-AZ-4315-01A-01D-1408-10,23,0,23


In [11]:
df_per_tx_sample_variant_type[
    (df_per_tx_sample_variant_type['hugo_symbol'] == 'TP53') &
    (df_per_tx_sample_variant_type['tumor_sample_barcode'] == 'TCGA-13-0755-01A-01W-0371-08')
]

,hugo_symbol,transcript_id,variant_classification,tumor_sample_barcode,gdc_count,mc3_count,diff
410281,TP53,ENST00000269305,Splice_Region,TCGA-13-0755-01A-01W-0371-08,2,0,2
573607,TP53,ENST00000269305,Silent,TCGA-13-0755-01A-01W-0371-08,0,1,-1


In [12]:
df_per_tx_sample = (
    df_per_tx_sample_variant_type
    .groupby(['hugo_symbol', 'transcript_id', 'tumor_sample_barcode'])
    .sum()
    .sort_values(['diff'], ascending=False)
    .reset_index()
)

In [13]:
df_per_tx_sample.head()

,hugo_symbol,transcript_id,tumor_sample_barcode,gdc_count,mc3_count,diff
0,TTN,ENST00000591111,TCGA-CA-6717-01A-11D-1835-10,68,0,68
1,TTN,ENST00000591111,TCGA-AA-A010-01A-01D-A17O-10,48,0,48
2,TTN,ENST00000591111,TCGA-AN-A046-01A-21W-A050-09,45,0,45
3,TTN,ENST00000591111,TCGA-AZ-4315-01A-01D-1408-10,44,0,44
4,TTN,ENST00000591111,TCGA-AA-A00N-01A-02D-A17O-10,40,0,40


In [14]:
df_per_tx = (
    df_per_tx_sample.groupby(['hugo_symbol', 'transcript_id'])
    .sum()
    .reset_index()
    .assign(absdiff = lambda df: df['diff'].abs())
    .sort_values(['absdiff'], ascending=False)
)

In [15]:
df_per_tx[df_per_tx['absdiff'] >= 150].sort_values(['hugo_symbol', 'mc3_count'])

,hugo_symbol,transcript_id,gdc_count,mc3_count,diff,absdiff
670,ADGRV1,ENST00000405460,216,0,216,216
1022,ALMS1,ENST00000613296,150,0,150,150
1385,APC,ENST00000257430,592,0,592,592
1386,APC,ENST00000457016,0,555,-555,555
3583,CCDC168,ENST00000322527,283,93,190,190
5180,CSMD1,ENST00000520002,252,0,252,252
5181,CSMD1,ENST00000537824,0,260,-260,260
5753,DCHS2,ENST00000623607,159,0,159,159
5751,DCHS2,ENST00000357232,0,154,-154,154
6157,DNAH11,ENST00000409508,244,0,244,244


In [17]:
# Export data to CSV
df_per_tx.to_csv('../variant_count_diff.per_tx.csv', index=False)
df_per_tx_sample.to_csv('../variant_count_diff.per_tx_sample.csv', index=False)
df_per_tx_sample_variant_type.to_csv('../variant_count_diff.per_tx_sample_type.csv', index=False)

In [18]:
df_per_tx[df_per_tx['hugo_symbol'].str.startswith('U2AF1')]

,hugo_symbol,transcript_id,gdc_count,mc3_count,diff,absdiff
24142,U2AF1L4,ENST00000412391,16,0,16,16
24140,U2AF1,ENST00000291552,4,13,-9,9
24141,U2AF1L4,ENST00000292879,0,9,-9,9


In [24]:
df_per_tx_sample[df_per_tx_sample['hugo_symbol'] == 'APC'].head()

,hugo_symbol,transcript_id,tumor_sample_barcode,gdc_count,mc3_count,diff
70,APC,ENST00000257430,TCGA-CA-6717-01A-11D-1835-10,9,0,9
109,APC,ENST00000257430,TCGA-AN-A046-01A-21W-A050-09,7,0,7
117,APC,ENST00000257430,TCGA-CM-6166-01A-11D-1650-10,7,0,7
150,APC,ENST00000257430,TCGA-AA-A010-01A-01D-A17O-10,7,0,7
252,APC,ENST00000257430,TCGA-SS-A7HO-01A-21D-A36X-10,5,0,5


In [20]:
pd.set_option('display.max_columns', None)

## TP53

Check if there is any flag for the variants of TP53.

In [21]:
sample = 'TCGA-BH-A0EE-01A-11W-A050-09'
gene = 'TP53'
df_gdc = pd.read_sql(
    '''
    SELECT 
        chromosome, start_position, end_position, 
        variant_classification, variant_type, 
        reference_allele, tumor_seq_allele1, tumor_seq_allele2,
        dbsnp_rs, sequencer,
        t_depth_per_caller, t_ref_count_per_caller, t_alt_count_per_caller,
        n_depth_per_caller, callers
    FROM gdc_grouped_callers
    WHERE hugo_symbol=? AND tumor_sample_barcode=?
    ''',
    conn,
    params=[gene, sample]
).sort_values(['start_position', 'end_position', "callers"])
df_mc3 = pd.read_sql(
    '''
    SELECT 
        chromosome, start_position, end_position, 
        variant_classification, variant_type, 
        reference_allele, tumor_seq_allele1, tumor_seq_allele2,
        dbsnp_rs,
        t_depth, t_ref_count, t_alt_count, n_depth,
        centers, ncallers
    FROM mc3_selected
    WHERE hugo_symbol=? AND tumor_sample_barcode=?
    ''',
    conn,
    params=[gene, sample]
)

In [22]:
df_gdc

,chromosome,start_position,end_position,variant_classification,variant_type,reference_allele,tumor_seq_allele1,tumor_seq_allele2,dbsnp_rs,sequencer,t_depth_per_caller,t_ref_count_per_caller,t_alt_count_per_caller,n_depth_per_caller,callers
1,chr17,7675079,7675079,Missense_Mutation,SNP,T,T,G,,Illumina Genome Analyzer II,"86,99","58,52","25,16","167,145",somaticsniper|varscan
0,chr17,7675079,7675079,Frame_Shift_Del,DEL,T,T,-,novel,Illumina Genome Analyzer II,109,76,32,145,varscan
2,chr17,7675085,7675085,Missense_Mutation,SNP,C,C,G,,Illumina Genome Analyzer II,"115,115,96","66,66,50","45,45,40","153,153,131",muse|somaticsniper|varscan


In [23]:
df_mc3

,chromosome,start_position,end_position,variant_classification,variant_type,reference_allele,tumor_seq_allele1,tumor_seq_allele2,dbsnp_rs,t_depth,t_ref_count,t_alt_count,n_depth,centers,ncallers
0,17,7675079,7675085,Frame_Shift_Del,DEL,TGGGGGC,TGGGGGC,GGGGGG,novel,67,37,30,69,PINDEL|MUSE*|SOMATICSNIPER*|VARSCANI*|INDELOCA...,7


## Exact Overlap

In [51]:
tx_id = 'ENST00000269305'
df_exact_overlap = pd.read_sql(
f'''\
WITH gdc_of_a_tx AS (
    SELECT 
        chromosome, start_position, end_position, tumor_sample_barcode,
        variant_classification, variant_type, 
        reference_allele, tumor_seq_allele1, tumor_seq_allele2,
        t_depth_per_caller, t_ref_count_per_caller, t_alt_count_per_caller,
        n_depth_per_caller, callers
    FROM gdc_grp_shared_samples
    WHERE transcript_id='{tx_id}'
), mc3_of_a_tx AS (
    SELECT 
        'chr' || chromosome AS chromosome, start_position, end_position, tumor_sample_barcode,
        variant_classification, variant_type, 
        reference_allele, tumor_seq_allele1, tumor_seq_allele2,
        t_depth, t_ref_count, t_alt_count, n_depth,
        centers, ncallers
    FROM mc3_selected
    WHERE transcript_id='{tx_id}'
) 
SELECT 
    start_position, end_position, tumor_sample_barcode, reference_allele, tumor_seq_allele2,
    g.tumor_seq_allele1 AS gdc_tumor_seq_allele1, 
    m.tumor_seq_allele1 AS mc3_tumor_seq_allele1,
    g.variant_classification AS gdc_variant_classification, 
    m.variant_classification AS mc3_variant_classification, 
    g.variant_type AS gdc_variant_type,
    m.variant_type AS mc3_variant_type,
    g.callers AS gdc_callers,
    m.centers AS mc3_callers, 
    m.ncallers AS mc3_ncallers
FROM gdc_of_a_tx g
LEFT JOIN mc3_of_a_tx m
    USING (tumor_sample_barcode, start_position, end_position, reference_allele, tumor_seq_allele2)
UNION ALL
SELECT 
    start_position, end_position, tumor_sample_barcode, reference_allele, tumor_seq_allele2,
    g.tumor_seq_allele1 AS gdc_tumor_seq_allele1, 
    m.tumor_seq_allele1 AS mc3_tumor_seq_allele1,
    g.variant_classification AS gdc_variant_classification, 
    m.variant_classification AS mc3_variant_classification, 
    g.variant_type AS gdc_variant_type,
    m.variant_type AS mc3_variant_type,
    g.callers AS gdc_callers,
    m.centers AS mc3_callers, 
    m.ncallers AS mc3_ncallers
FROM mc3_of_a_tx m
LEFT JOIN gdc_of_a_tx g
    USING (tumor_sample_barcode, start_position, end_position, reference_allele, tumor_seq_allele2)
WHERE gdc_tumor_seq_allele1 IS NULL
''', conn)

In [52]:
df_exact_overlap = df_exact_overlap.sort_values(['start_position'])

In [55]:
df_exact_overlap = df_exact_overlap[~df_exact_overlap.duplicated()]

In [35]:
df_exact_overlap.to_csv('../tp53_overlap.csv')

In [58]:
df_exact_overlap.head()

,start_position,end_position,tumor_sample_barcode,reference_allele,tumor_seq_allele2,gdc_tumor_seq_allele1,mc3_tumor_seq_allele1,gdc_variant_classification,mc3_variant_classification,gdc_variant_type,mc3_variant_type,gdc_callers,mc3_callers,mc3_ncallers
354,7669353,7669353,TCGA-29-1691-01A-01W-0633-09,T,C,T,None,3'UTR,None,SNP,None,mutect,None,None
751,7669464,7669464,TCGA-AD-6889-01A-11D-1924-10,C,-,C,None,3'UTR,None,DEL,None,mutect,None,None
1059,7669511,7669511,TCGA-D8-A1XK-01A-21D-A14K-09,T,C,T,T,3'UTR,3'UTR,SNP,SNP,muse,MUTECT|MUSE,2
533,7669624,7669624,TCGA-61-2113-01A-01W-0722-08,C,T,C,None,Silent,None,SNP,None,mutect,None,None
284,7670664,7670664,TCGA-24-1843-01A-01W-0639-09,C,A,A,C,Nonsense_Mutation,Nonsense_Mutation,SNP,SNP,varscan,SOMATICSNIPER|RADIA|MUTECT|MUSE|VARSCANS,5


In [71]:
def grp_fun(df):
    gdc_called = df['gdc_callers'].notnull()
    mc3_called = df['mc3_callers'].notnull()
    return pd.DataFrame([[ 
        (gdc_called & mc3_called).sum(),  
        gdc_called.sum() - (gdc_called & mc3_called).sum(),
        mc3_called.sum() - (gdc_called & mc3_called).sum()
    ]], columns=['shared', 'only_in_gdc', 'only_in_mc3'])
df_overlap_grp = df_exact_overlap.groupby(['start_position', 'end_position']).apply(grp_fun).reset_index()

In [75]:
df_overlap_grp[df_overlap_grp['only_in_gdc'] > 0]

,start_position,end_position,level_2,shared,only_in_gdc,only_in_mc3
0,7669353,7669353,0,0,1,0
1,7669464,7669464,0,0,1,0
3,7669624,7669624,0,0,1,0
23,7673243,7673243,0,0,1,0
56,7673755,7673756,0,0,1,0
57,7673756,7673757,0,0,1,0
66,7673775,7673776,0,0,1,0
75,7673783,7673796,0,0,1,0
78,7673788,7673788,0,8,1,0
86,7673801,7673810,0,0,1,0


In [76]:
df_overlap_grp[df_overlap_grp['only_in_mc3'] > 0]

,start_position,end_position,level_2,shared,only_in_gdc,only_in_mc3
68,7673776,7673778,0,0,0,1
88,7673803,7673803,0,19,1,1
90,7673806,7673806,0,12,0,1
95,7673813,7673814,0,0,0,1
112,7674185,7674186,0,0,0,1
208,7674957,7674959,0,2,0,1
224,7675079,7675085,0,0,0,1
260,7675143,7675147,0,0,0,1
268,7675149,7675182,0,0,0,1
284,7675182,7675182,0,1,1,1


In [78]:
df_overlap_grp.to_csv('../tp53_overlap.csv', index=False)